## Imports

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## Constants

In [ ]:
HEADER_LEN = 6
KEEP = 1
COLORS = ['#1f77b4',
          '#ff7f0e',
          '#2ca02c',
          '#d62728',
          '#9467bd',
          '#8c564b',
          '#e377c2',
          '#7f7f7f',
          '#bcbd22',
          '#17becf',
          '#1a55FF']
CURRENT_WORKING_DIR = os.getcwd()
print(CURRENT_WORKING_DIR)

#todo make dynamic
os.chdir('D:/Switchdrive/Uni/Thesis/code/HuberNicolas/MTDPolicy/')



CAT = {
    'usr': 'total-cpu-usage',
    'sys': 'total-cpu-usage',
    'idl': 'total-cpu-usage',
    'wai': 'total-cpu-usage',
    'hiq': 'total-cpu-usage',
    'siq': 'total-cpu-usage',

    'used' : 'memory usage',
    'buff' : 'memory usage',
    'cache' : 'memory usage',
    'free' : 'memory usage',

    'files': 'filesystem',
    'inodes' : 'fileystem',

    'read' : 'dsk/total',
    'writ' : 'dsk/total',
    'reads' : 'dsk/total',
    'writs' : 'dsk/total',

    'recv' : 'net/eth0',
    'send' : 'net/eth0',

    'lis' : 'tcp sockets',
    'act' : 'tcp sockets',
    'syn' : 'tcp sockets',
    'tim' : 'tcp sockets',
    'clo' : 'tcp sockets',

    'int' : 'system',
    'csw' : 'system',

    'run' : 'procs',
    'blk' : 'procs',
    'new' : 'procs'
}

CAT_NAMES = ['total-cpu-usage', 'memory-usage', 'filesystem', 'dsk-total1', 'dsk-total2', 'net-eth0', 'tcp sockets', 'system', 'procs',]

COL_NAMES = [
    # total-cpu-usage
    'usr: CPU usage by user processes [%]', 
    'sys: CPU usage by system processes [%]',
    'idl: Number of idle processes [#]',
    'wai: Number of waiting processes [#]',
    'hiq: Number of hard interrupts [#]',
    'siq: Number of soft interrupts [#]',
    # memory usage
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    # https://linuxtect.com/linux-dstat-command-monitor-linux-performance-and-resource-usage/
    # https://www.thegeekdiary.com/understanding-proc-meminfo-file-analyzing-memory-utilization-in-linux/
    # /proc/meminfo
    'used: Amount of used memory [Bytes]', # / 1024 = KB
    'buff: Amount of buffered memory [Bytes]', # / 1024 / 1024 = MB
    'cache: Amount of cached memory [Bytes]',
    'free: Amount of free memory [Bytes]',

    # filesystem
    # https://www.linuxquestions.org/questions/linux-general-1/proc-sys-fs-file-nr-57646/
    # /proc/sys/fs/file-nr
    'files: Number of allocated file handles [#]',
    'inodes: Number of used file handles [#]',

    # dsk/total
    'read: Amount of read bytes on disk [Bytes]', # not 100% sure
    'writ: Amount of written bytes on disk [Bytes]', # / 1024 = KB

    # dsk/total
    'reads: Number of read operations on disk [#]',
    'writs: Number of write operations on disk [#]',
    
    # net/eth0
    'recv: Amount of received bytes on eth0 [B]', # /1024 = k
    'send: Amount of received bytes on eth0 [B]',

    # tcp sockets
    # http://karunsubramanian.com/network/what-is-syn_sent-socket-status/
    # /proc/net/tcp
    # /proc/net/tcp6
    'lis: Number of TCP connections with status "listening" [#]', 
    'act: Number of TCP connections with status "established" (active) [#]',
    'syn: Number of TCP connections with status "syn_sent", "syn_receive" or "last_ack" [#]',
    'tim: Number of TCP connections with status "waiting"[#]',
    'clo: Number of TCP connections with status "fin-wait1/2", "close/_wait" or "closign" (closed) [#]',

    # system
    # https://www.tecmint.com/dstat-monitor-linux-server-performance-process-memory-network/
    'int: Number of interrupts [#]', 
    'csw: Number of ontext switches [#]', 

    # procs
    # /proc/stat
    'run: Number of processes with status "running" [#]', 
    'blk: Number of processes with status "blocked" [#]',
    'new: Number of processes with status "new" [#]' 
]
PERIOD = 60

## Load csv files

### Loading

In [ ]:
os.chdir('./data/csv')
healthy_files = glob.glob('healthy\*.csv')
infected_files = glob.glob('infected\*.csv')

# create dataframes for each .csv
dfs = []
header = None
for f in healthy_files:
    # create header
    if header is None:
        header = pd.read_csv(f, skiprows=HEADER_LEN)
        header = list(header.columns.values)
    # read and append
    dff = pd.read_csv(f, skiprows = HEADER_LEN + 1, header = None) # read csv
    dff = dff.iloc[:, :-1] # remove last row
    dff.columns = header # append header
    dfs.append(dff) # append to list of dfs
    
# append all dataframes dff to one dataframe df
df_healthy = pd.concat(dfs, ignore_index=True)


# create dataframes for each .csv
dfs = []
header = None
for f in infected_files:
    # create header
    if header is None:
        header = pd.read_csv(f, skiprows=HEADER_LEN)
        header = list(header.columns.values)
    # read and append
    dff = pd.read_csv(f, skiprows = HEADER_LEN + 1, header = None) # read csv
    #dff = dff.iloc[:, :-1] # remove last row
    dff.columns = header # append header
    dfs.append(dff) # append to list of dfs
    
# append all dataframes dff to one dataframe df
df_infected = pd.concat(dfs, ignore_index=True)

#print(df_healthy)
#print(df_infected)
#df_healthy.to_csv('healthy.csv', index=False, header=True)
#df_infected.to_csv('infected.csv', index=False, header=True)



# fix year
df_healthy['time'] = '2022-' + df_healthy['time'].astype(str)
df_healthy['time'] = pd.to_datetime(df_healthy['time'],format= '%Y-%d-%m %H:%M:%S')
# sort and reindex
df_healthy = df_healthy.sort_values(by='time')
df_healthy = df_healthy.iloc[::KEEP] # only take every KEEPth-entry
df_healthy = df_healthy.reset_index(drop=True)
df_healthy.set_index('time', inplace=True)

# fix year
df_infected['time'] = '2022-' + df_infected['time'].astype(str)
df_infected['time'] = pd.to_datetime(df_infected['time'],format= '%Y-%d-%m %H:%M:%S')
# sort and reindex
df_infected = df_infected.sort_values(by='time')
df_infected = df_infected.iloc[::KEEP] # only take every KEEPth-entry
df_infected = df_infected.reset_index(drop=True)
df_infected.set_index('time', inplace=True)


# save file
startDate = df_healthy.iloc[0].name.strftime('%Y-%m-%d')
startTime = df_healthy.iloc[0].name.strftime('%X')
endDate = df_healthy.iloc[-1].name.strftime('%Y-%m-%d')
endTime = df_healthy.iloc[-1].name.strftime('%X')
name = "{}-{}_{}-{}({}).csv".format(startDate.replace('-',''), startTime.replace(':',''), endDate.replace('-',''), endTime.replace(':',''), str(df_healthy.shape[0]))

df_healthy.to_csv('healthy.csv', index=True, header=True)

# save file
startDate = df_infected.iloc[0].name.strftime('%Y-%m-%d')
startTime = df_infected.iloc[0].name.strftime('%X')
endDate = df_infected.iloc[-1].name.strftime('%Y-%m-%d')
endTime = df_infected.iloc[-1].name.strftime('%X')
name = "{}-{}_{}-{}({}).csv".format(startDate.replace('-',''), startTime.replace(':',''), endDate.replace('-',''), endTime.replace(':',''), str(df_infected.shape[0]))

df_infected.to_csv('infected.csv', index=True, header=True)


print(df_healthy.columns)





In [ ]:
WINDOWS = [1, 5, 6, 10, 20, 30, 60, 120]
WINDOWS = [30, 60, 120]
# todo fix reset index
df_healthy.reset_index(inplace=True)
df_infected.reset_index(inplace=True)

cols = df_healthy.columns.values.tolist()
cols = cols[2:] # remove index and time col
print(cols)

for i in cols:
    str = i + '-mean'
    for window in WINDOWS:

        df_healthy[str]  = df_healthy[i].rolling(window).mean()[window-1::window]
        df_healthy[str].bfill(inplace = True)
        df_infected[str]  = df_infected[i].rolling(window).mean()[window-1::window]
        df_infected[str].bfill(inplace = True)
        
        
        plt.figure(dpi=200)
        plt.title(label=str)
        df_healthy[str].plot(marker="o", color=COLORS[1], linestyle='-', linewidth=0.5, label= str + " healthy")
        df_infected[str].plot(marker="o", color=COLORS[2], linestyle='-', linewidth=0.5, label= str + " infected")
        plt.xlabel('Timestamp [hh:mm:ss] from {}'.format(startDate))
        plt.ylabel(str)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 0))
        plt.tight_layout()
        plt.title(str)
        plt.savefig(str+'{}'.format(window), facecolor='white', transparent=False)   
        plt.show()

for i in cols:
    str = i + '-max'
    for window in WINDOWS:

        df_healthy[str]  = df_healthy[i].rolling(window).max()[window-1::window]
        df_healthy[str].bfill(inplace = True)
        df_infected[str]  = df_infected[i].rolling(window).max()[window-1::window]
        df_infected[str].bfill(inplace = True)
        
        
        plt.figure(dpi=200)
        plt.title(label=str)
        df_healthy[str].plot(marker="o", color=COLORS[1], linestyle='-', linewidth=0.5, label= str + " healthy")
        df_infected[str].plot(marker="o", color=COLORS[2], linestyle='-', linewidth=0.5, label= str + " infected")
        plt.xlabel('Timestamp [hh:mm:ss] from {}'.format(startDate))
        plt.ylabel(str)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 0))
        plt.tight_layout()
        plt.title(str)
        plt.savefig(str+'{}'.format(window), facecolor='white', transparent=False)   
        plt.show()

for i in cols:
    str = i + '-min'
    for window in WINDOWS:

        df_healthy[str]  = df_healthy[i].rolling(window).min()[window-1::window]
        df_healthy[str].bfill(inplace = True)
        df_infected[str]  = df_infected[i].rolling(window).min()[window-1::window]
        df_infected[str].bfill(inplace = True)
        
        
        plt.figure(dpi=200)
        plt.title(label=str)
        df_healthy[str].plot(marker="o", color=COLORS[1], linestyle='-', linewidth=0.5, label= str + " healthy")
        df_infected[str].plot(marker="o", color=COLORS[2], linestyle='-', linewidth=0.5, label= str + " infected")
        plt.xlabel('Timestamp [hh:mm:ss] from {}'.format(startDate))
        plt.ylabel(str)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 0))
        plt.tight_layout()
        plt.title(str)
        plt.savefig(str+'{}'.format(window), facecolor='white', transparent=False)   
        plt.show()


'''
for window in WINDOWS:
    df_healthy['usr-mean']  = df_healthy['usr'].rolling(window).mean()[window-1::window]
    df_healthy['usr-mean'].bfill(inplace = True)
    df_infected['usr-mean']  = df_infected['usr'].rolling(window).mean()[window-1::window]
    df_infected['usr-mean'].bfill(inplace = True)
    
    
    plt.figure(dpi=200)
    df_healthy['usr-mean'].plot(marker="o", color=COLORS[1], linestyle='-', linewidth=0.5)
    df_infected['usr-mean'].plot(marker="o", color=COLORS[2], linestyle='-', linewidth=0.5)  
    plt.show()
'''
